# LSTM - Time-series Forecasting (Stocks)


There are several time-series forecasting techniques like auto regression (AR) models, moving average (MA) models, Holt-winters, ARIMA etc., to name a few. 

So, what is the need for yet another model like LSTM-RNN to forecast time-series? 

RNN’s (LSTM’s) are pretty good at extracting patterns in input feature space, where the input data spans over long sequences. Given the gated architecture of LSTM’s that has this ability to manipulate its memory state, they are ideal for such problems.

LSTMs can almost seamlessly model problems with multiple input variables. 

All we need is a 3D input vector [N-examples(time serie), T-timesteps(moving window), F-features(stock)] that needs to be fed into the input shape of the LSTM. 

So long as we figure out a way to convert all our input variables to be represented in a 3D vector form, we are good use LSTM. This adds a great benefit in time series forecasting, where classical linear methods can be difficult to adapt to multivariate or multiple input forecasting problems 

(A side note here for multivariate forecasting — keep in mind that when we use multivariate data for forecasting, then we also need “future multi-variate” data to predict the future outcome!)

LSTM’s offer lot of flexibility in modelling the problem, like;

- Using several combinations of seq2seq LSTM models to forecast time-series:
 — many to one model(useful when we want to predict at the current timestep given all the previous inputs), 
 - many to many model (useful when we want to predict multiple future time steps at once given all the previous inputs) and several other variations on these. 
 
- Customizing several things for example:
 — the size of look-back window to predict at the current step, 
 - the number of time steps we want to predict into the future, feeding the current prediction back into the window to make prediction at the next time step (this technique also known as moving-forward window) and so on.

On the other hand, LSTM’s need lots of data, multiple hyper-parameters to be tuned etc., LSTM’s are supposedly not good at auto-regression type of series.


fig 1: Plot of entire sine wave data
A brief about the overall approach before we dive deep into details —
use a moving forward window of size 50, which means we will use the first 50 data points as out input X to predict y1 — 51st data point. Next, we will use the window between 1 to 51 data points as input X to predict y2 i.e., the 52nd data point and so on…Here is the plot of first 50 data points —

2. use a two layered LSTM architecture coupled with a dense output layer to make a prediction.

3. We will look at couple of approaches to predict the output — a.) Forecasting step by step on the test data set, b.) Feed the previous prediction back into the input window by moving it one step forward and then predict at the current time step.
Now lets dive into the details —
Data preparation —
Normalizing the data using minmax scaler ( refer below code snippet)

2. Fix the moving window size to be 50. For this purpose we use pandas shift function that shifts the entire column by the number we specify. In the below code snippet, we shifted the column up by 1 (hence used -1. If we want to shift it down by 1, we will have to use +1) and then concatenate that to the original data.

I tried to illustrate this on a toy data set below as to how the above for loop works for a window_size of 3.

Note — we dropped all the rows that contain the Nan values in the above code snippet.
If you look at the toy data set closely, you can observe that this models the input data in the fashion we want to input into the LSTM. The last column in the above table becomes the target y and the first three columns become our input x1,x2 and x3 features. If you are familiar with using LSTM for NLP, then you can look at this as a fixed sequence of length 3 of sentence containing 3 words each and we are tasked with predicting the 4th word.
3.Preparing the 3D input vector for the LSTM. Remember, the input vector for LSTM is 3D array: (num_samples, num_time_steps, num_features). In this case we have num of time steps = 50 and num_features = 1 (Extending the same analogy we saw in the previous point, that I found very useful in understanding why the input shape has to be like this — lets say, we have 50 words in one sentence and each word is represented by a word vector. So we need 50 time steps to go through each word vector in the sentence as an input to the LSTM at each time step. There is one sentence per observation and hence num_features = 1. Like this, we need to iterate over all the sentences in the train data to extract the pattern between the words in all sentences. This is exactly what we want here in the time series forecast as well — we want to identify all the patterns that exist between each of the previous values in the window to predict the current time step!)
Model Architecture —
Below is the model architecture used that is quite self-explanatory—(Its a double stacked LSTM layers with the output from the first LSTM at each time step is being fed to the second LSTM)

Model architecture
Making predictions —
Predicting step by step on the test data ( refer to the below code snippet). This is quite straight forward. Given all the learned parameters from train data, we are using them to predict on all the test sequences one at a time.

The plot of predictions vs actuals almost overlap with each other to the extent that we cannot distinguish the blue curve and red curve in the below plot.

However, the above is usually not a realistic way in which predictions are done, as we will not have all the future window sequences available with us.
2. So, if we want to predict multiple time steps into the future, then a more realistic way is to predict one time step at a time into the future and feed that prediction back into the input window at the rear while popping out the first observation at the beginning of the window (so that the window size remains same). Refer to the below code snippet that does this part — (the comments in the code are self explanatory if you go through the code in my github link that I mentioned above ) —

Using this prediction model, the results are plotted below —

As can be seen, quite understandably, the farther we try to predict in time, more the error at each time-step that builds up on the previous predicted error. However, the function still behaves like a dampening sine-wave! As I said earlier, this is more realistic modelling of any time series problem since we would not have all the future sequences in hand with us.
This code can very well be extended to predicting any time series in general. Note that you may need to take care of other aspects of data preparation like de-trending the series, differencing to stationarize the data and so on before it is fed to LSTM to forecast.

### Setup notebook

In [ ]:
# Importing the libraries
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mymods.lauthom import *

### Data

In [ ]:
get_path('_data', 'Google_Stock_Price_Train')

In [ ]:
# Importing the training set
dataset_train = pd.read_csv('../../_data/Google_Stock_Price_Train.csv')
dataset_train.head()

In [ ]:
training_set = dataset_train.loc[:, 'Open'].values

In [ ]:
training_set = dataset_train.loc[:, ['Open']].values
print(training_set.shape)

_ = plt.plot(training_set)

### Feature Scaling

Scale to range 0-1

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

_ = plt.plot(training_set_scaled)

### Creating a data structure with N_TIMESTEPS in columns and 1 output

Make rolling window by shifting 'Open' 1 to N timesteps;
 - each feature(column) is the previous feature shifted 1 timestep
 - each example(row) is N features representing a window of 70 timesteps(trading days)
 
Last N rows with NaN's should be deleted.

In [ ]:
# rolling trading days
N_TIMESTEPS = 70
N = training_set_scaled.shape[0]
N

In [ ]:
X_train, y_train = [], []

# build a list of timesteps-arrays
for i in range(N_TIMESTEPS, N):
    X_train.append(training_set_scaled[i - N_TIMESTEPS:i, 0])
    y_train.append(training_set_scaled[i, 0])

# list to ndarray    
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping - add dimension
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)
_ = plt.plot(X_train[:, 1])

## RNN - Keras

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
class RNN():
    """RNN regression model architecture using Keras"""
    def __init__(self, input_shape, layer_neurons, dropouts):
        
        # Initialising the RNN
        self.rnn = Sequential()

        # Add first layer[0] the LSTM layers and some Dropout regularisation
        self.add_lstm(layer_neurons[0], return_sequences=True, input_shape=input_shape)
        self.add_dropout(dropouts[0])
        
        # Add hidden layers[1:-1]
        for c, n in enumerate(layer_neurons[1:-1], 1):
            self.add_lstm(n, return_sequences=True)
            self.add_dropout(dropouts[c])
       
        # Add last hidden layer[-1]
        self.add_lstm(layer_neurons[-1])
        self.add_dropout(dropouts[-1])
        
        # Add output layer
        self.add_dense(1)
        
    def add_lstm(self, n, **kwargs):
        return self.rnn.add(LSTM(units=n, **kwargs))
    
    def add_dropout(self, d):
        return self.rnn.add(Dropout(d))
    
    def add_dense(self, d):
        return self.rnn.add(Dense(units=d))

    def compiler(self, **kwargs):
        return self.rnn.compile(**kwargs)

    def fit(self, x, y, **kwargs):
        return self.rnn.fit(x, y, **kwargs)

    def predict(self, x):
        return self.rnn.predict(x)

### Train model

In [ ]:
# set hyperparameters
N_EPOCH = 5
N_BATCH = 32
layer_neurons = (50, 50, 50, 50)
dropouts = (0, 0.5, 0, 0)

# compile and fit RNN
rnn = RNN((X_train.shape[1], 1), layer_neurons, dropouts)

rnn.compiler(optimizer='adam', loss='mean_squared_error')

history = rnn.fit(X_train, y_train, epochs=N_EPOCH, batch_size=N_BATCH)

In [ ]:
# dir(history)
history.model

### Test data (2017)

In [ ]:
dataset_test = pd.read_csv('../../_data/Google_Stock_Price_Test.csv')
dataset_test.info()
dataset_test['Open'].head()

### Preprocess testset

In [ ]:
# get scope: last N_TIMESTEPS from train set + test set TIMESTEPS
inputs = pd.concat((dataset_train.loc[-N_TIMESTEPS:, 'Open'], dataset_test.loc[:, 'Open']), axis=0).values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)  # use earlier defined sc transform
inputs.shape
N_TEST_TIMESTEPS = len(inputs) # N + 20

# N lists(Series)
X_test = [inputs[i-N_TIMESTEPS:i] for i in range(N_TIMESTEPS, N_TEST_TIMESTEPS)]

# list to ndarray 3D array     
X_test = np.array(X_test)
# X_test = X_test[..., np.newaxis] # if above used [inputs[i-N_TIMESTEPS:i, 0]
assert X_test.shape == (1208, 70, 1)

### Predict

Getting the predicted stock price of 2017.

In [ ]:
predicted_stock_price = rnn.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)[-20:]
predicted_stock_price.shape

In [ ]:
real_stock_price = dataset_test.loc[:, 'Open'].values
real_stock_price.shape

### Visualise

In [ ]:
_ = plt.plot(real_stock_price, color='red', label='Real Google Stock Price')
_ = plt.plot(predicted_stock_price, color='blue', label='Predicted Google Stock Price')
_ = plt.title('Google Stock Price Prediction')
_ = plt.xlabel('Time')
_ = plt.ylabel('Google Stock Price')
_ = plt.legend()
_ = plt.show()